# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method.

In [308]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

In [309]:
data_set = data_set
labels = labels
test_data_set = test_data_set
test_labels = test_labels
unique_labels = unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [310]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [311]:
def build_classifiers():

    # fill this part
    linear_regression = LinearRegression()
    linear_regression.fit(data_set, labels)
    # decision_tree = DecisionTreeClassifier(ccp_alpha= 0.01, random_state=21)
    # decision_tree.fit(data_set, labels)

    neighbors = KNeighborsClassifier(n_neighbors=3)
    neighbors.fit(data_set, labels)
    naive_bayes = GaussianNB()
    naive_bayes.fit(data_set, labels)

    linear_svm = SVC(kernel='linear', random_state=21)
    linear_svm.fit(data_set, labels)

    # naive_bayes = GaussianNB()
    # naive_bayes.fit(data_set, labels)

    discriminant_analysis = QuadraticDiscriminantAnalysis()
    discriminant_analysis.fit(data_set, labels)

    return neighbors, naive_bayes, linear_svm

In [312]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = GaussianNB()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [313]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

1.0


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [314]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set
def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

def calculate_accuracy_vector(predicted, test_labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == test_labels[i]:
            result.append(0)
        else:
            result.append(1)
    return result

Fill the two functions below:

In [315]:

def set_new_weights(model):
    predicted = model.predict(test_set)
    I = np.array(calculate_accuracy_vector(predicted, test_labels))  # Convert to NumPy array
    weights = (1 + I) / np.sum(1 + I)

    return weights

Train the classifier with the code below:

In [316]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(100, dimension, labels)

[0.00066313 0.00132626 0.00066313 0.00132626 0.00132626 0.00066313
 0.00066313 0.00132626 0.00066313 0.00132626 0.00132626 0.00132626
 0.00066313 0.00066313 0.00066313 0.00066313 0.00066313 0.00132626
 0.00132626 0.00132626 0.00132626 0.00066313 0.00066313 0.00066313
 0.00066313 0.00066313 0.00132626 0.00132626 0.00066313 0.00132626
 0.00066313 0.00066313 0.00132626 0.00066313 0.00132626 0.00066313
 0.00132626 0.00132626 0.00066313 0.00132626 0.00066313 0.00132626
 0.00132626 0.00132626 0.00132626 0.00066313 0.00066313 0.00066313
 0.00132626 0.00066313 0.00132626 0.00066313 0.00132626 0.00132626
 0.00066313 0.00132626 0.00132626 0.00066313 0.00132626 0.00132626
 0.00132626 0.00066313 0.00066313 0.00132626 0.00132626 0.00066313
 0.00066313 0.00066313 0.00066313 0.00066313 0.00132626 0.00066313
 0.00066313 0.00066313 0.00132626 0.00132626 0.00132626 0.00132626
 0.00066313 0.00066313 0.00066313 0.00132626 0.00132626 0.00132626
 0.00132626 0.00066313 0.00066313 0.00132626 0.00066313 0.0013

Set the validation data set:

In [317]:
validate_x, validate_label = generate_data(100, dimension, labels)

print(validate_x, validate_label)

[[0.40993875 0.3757303 ]] [1]


Fill the prediction code:

In [318]:
def get_prediction(x):
    weighted_votes = np.zeros(labels)  # Initialize an array for weighted votes
    for alpha, classifier in zip(alphas, classifiers):
        prediction = classifier.predict(x)  # Get the prediction from the classifier
        weighted_votes[prediction[0]] += alpha  # Add the weighted vote for the predicted class
    return np.argmax(weighted_votes)  # Return the class with the highest weighted vote

Test it:

In [319]:
prediction = get_prediction(validate_x)

print(prediction)

print(accuracy_score(validate_label, prediction))

Predicted: 0
True label: 1
